In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('homepage-experiment-data.csv')

In [3]:
data.describe()

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
count,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000
mean,15.000000,1615.551724,260.482759,24.482759,1632.62069,294.758621,25.241379
std,8.514693,116.308268,28.338037,13.873461,113.02636,22.404807,13.762410
min,1.000000,1457.000000,223.000000,1.000000,1458.00000,256.000000,1.000000
25%,8.000000,1529.000000,240.000000,12.000000,1555.00000,279.000000,20.000000
50%,15.000000,1602.000000,254.000000,30.000000,1606.00000,290.000000,29.000000
75%,22.000000,1700.000000,276.000000,34.000000,1728.00000,300.000000,36.000000
max,29.000000,1822.000000,331.000000,42.000000,1861.00000,349.000000,44.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Day                   29 non-null     int64
 1   Control Cookies       29 non-null     int64
 2   Control Downloads     29 non-null     int64
 3   Control Licenses      29 non-null     int64
 4   Experiment Cookies    29 non-null     int64
 5   Experiment Downloads  29 non-null     int64
 6   Experiment Licenses   29 non-null     int64
dtypes: int64(7)
memory usage: 1.7 KB
